# How to Do Deep Learning on Graphs with Graph Convolutional Networks
## Part 2: Semi-Supervised Learning with Spectral Graph Convolutions
This notebook accompanies my Medium article with the above title for readers to try out and explore graph convolutional networks for themselves. You can find the article [here](https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-62acf5b143d0). To run the notebook, install the packages specified in the accompanying ```requirements.txt``` file.

#  Load Karate Club

In [1]:
from collections import namedtuple
from networkx import read_edgelist, set_node_attributes
from pandas import read_csv, Series
from numpy import array

DataSet = namedtuple(
    'DataSet',
    field_names=['X_train', 'y_train', 'X_test', 'y_test', 'network']
)

def load_karate_club():
    network = read_edgelist(
        'karate.edgelist',
        nodetype=int)

    attributes = read_csv(
        'karate.attributes.csv',
        index_col=['node'])

    for attribute in attributes.columns.values:
        set_node_attributes(
            network,
            values=Series(
                attributes[attribute],
                index=attributes.index).to_dict(),
            name=attribute
        )

    X_train, y_train = map(array, zip(*[
        ([node], data['role'] == 'Administrator')
        for node, data in network.nodes(data=True)
        if data['role'] in {'Administrator', 'Instructor'}
    ]))
    X_test, y_test = map(array, zip(*[
        ([node], data['community'] == 'Administrator')
        for node, data in network.nodes(data=True)
        if data['role'] == 'Member'
    ]))
    
    return DataSet(
        X_train, y_train,
        X_test, y_test,
        network)

In [2]:
from networkx import to_numpy_matrix, degree_centrality, betweenness_centrality, shortest_path_length
import mxnet.ndarray as nd

zkc = load_karate_club()

A = to_numpy_matrix(zkc.network)
A = nd.array(A)

X_train = zkc.X_train.flatten()
y_train = zkc.y_train
X_test = zkc.X_test.flatten()
y_test = zkc.y_test

# Layer Implementations

In [3]:
from mxnet.gluon import HybridBlock
from mxnet.gluon.nn import Activation
import mxnet.ndarray as nd

class SpectralRule(HybridBlock):
    def __init__(self, A, in_units, out_units, activation='relu', **kwargs):
        super().__init__(**kwargs)
        I = nd.eye(*A.shape)
        A_hat = A.copy() + I

        D = nd.sum(A_hat, axis=0)
        D_inv = D**-0.5
        D_inv = nd.diag(D_inv)

        A_hat = D_inv * A_hat * D_inv
        
        self.in_units, self.out_units = in_units, out_units
        
        with self.name_scope():
            self.A_hat = self.params.get_constant('A_hat', A_hat)
            self.W = self.params.get(
                'W', shape=(self.in_units, self.out_units)
            )
            if activation == 'identity':
                self.activation = lambda X: X
            else:
                self.activation = Activation(activation)

    def hybrid_forward(self, F, X, A_hat, W):
        aggregate = F.dot(A_hat, X)
        propagate = self.activation(
            F.dot(aggregate, W))
        return propagate

In [4]:
class LogisticRegressor(HybridBlock):
    def __init__(self, in_units, **kwargs):
        super().__init__(**kwargs)
        with self.name_scope():
            self.w = self.params.get(
                'w', shape=(1, in_units)
            )

            self.b = self.params.get(
                'b', shape=(1, 1)
            )

    def hybrid_forward(self, F, X, w, b):
        # Change shape of b to comply with MXnet addition API
        b = F.broadcast_axis(b, axis=(0,1), size=(34, 1))
        y = F.dot(X, w, transpose_b=True) + b

        return F.sigmoid(y)

# Models

In [5]:
from mxnet.gluon.nn import HybridSequential, Activation
from mxnet.ndarray import array
from mxnet.initializer import One, Uniform, Xavier
from mxnet.gluon.loss import SigmoidBinaryCrossEntropyLoss

def build_features(A, X):
    hidden_layer_specs = [(4, 'tanh'), (2, 'tanh')] # Format: (units in layer, activation function)
    in_units = in_units=X.shape[1]
  
    features = HybridSequential()
    with features.name_scope():
        for i, (layer_size, activation_func) in enumerate(hidden_layer_specs):
            layer = SpectralRule(
                A, in_units=in_units, out_units=layer_size, 
                activation=activation_func)
            features.add(layer)

            in_units = layer_size
    return features, in_units

def build_model(A, X):
    model = HybridSequential()
    hidden_layer_specs = [(4, 'tanh'), (2, 'tanh')]
    in_units = in_units=X.shape[1]

    with model.name_scope():
        features, out_units = build_features(A, X)
        model.add(features)

        classifier = LogisticRegressor(out_units)
        model.add(classifier)

    model.hybridize()
    model.initialize(Uniform(1))

    return model, features

## Model 1: Identity Matrix as Features

In [6]:
X_1 = I = nd.eye(*A.shape)
model_1, features_1 = build_model(A, X_1)
model_1(X_1)


[[0.5106776 ]
 [0.5095115 ]
 [0.51096225]
 [0.51542187]
 [0.52302504]
 [0.512526  ]
 [0.511652  ]
 [0.50208426]
 [0.5050117 ]
 [0.51883554]
 [0.5406651 ]
 [0.49158806]
 [0.5110878 ]
 [0.4936631 ]
 [0.5026252 ]
 [0.48504698]
 [0.5104648 ]
 [0.51194   ]
 [0.5220448 ]
 [0.5095117 ]
 [0.50140274]
 [0.50985765]
 [0.5303379 ]
 [0.51043373]
 [0.50124794]
 [0.52045095]
 [0.50941336]
 [0.49507195]
 [0.5021959 ]
 [0.50682354]
 [0.5085804 ]
 [0.51637703]
 [0.5098626 ]
 [0.5153003 ]]
<NDArray 34x1 @cpu(0)>

## Model 2: Distance to Administrator and Instructor as Additional Features

In [7]:
X_2 = nd.zeros((A.shape[0], 2))
node_distance_instructor = shortest_path_length(zkc.network, target=33)
node_distance_administrator = shortest_path_length(zkc.network, target=0)

for node in zkc.network.nodes():
    X_2[node][0] = node_distance_administrator[node]
    X_2[node][1] = node_distance_instructor[node]

In [8]:
X_2 = nd.concat(X_1, X_2)
model_2, features_2 = build_model(A, X_2)
model_2(X_2)


[[0.45661393]
 [0.45822805]
 [0.456899  ]
 [0.457218  ]
 [0.45603466]
 [0.4640951 ]
 [0.4631094 ]
 [0.45584652]
 [0.4608328 ]
 [0.47084248]
 [0.4593098 ]
 [0.4513544 ]
 [0.46219462]
 [0.47803804]
 [0.46955827]
 [0.49520862]
 [0.46266046]
 [0.4590963 ]
 [0.49145314]
 [0.46135917]
 [0.48991072]
 [0.4575545 ]
 [0.48936   ]
 [0.4587967 ]
 [0.46603325]
 [0.47440898]
 [0.48096892]
 [0.47632974]
 [0.47681022]
 [0.47006816]
 [0.47394764]
 [0.46878797]
 [0.4676332 ]
 [0.49637428]]
<NDArray 34x1 @cpu(0)>

# Train and Test Models

In [9]:
%time
from mxnet import autograd
from mxnet.gluon import Trainer
from mxnet.ndarray import sum as ndsum
import numpy as np

def train(model, features, X, X_train, y_train, epochs):
    cross_entropy = SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)
    trainer = Trainer(model.collect_params(), 'sgd', {'learning_rate': 0.001, 'momentum': 1})

    feature_representations = [features(X).asnumpy()]

    for e in range(1, epochs + 1):
        cum_loss = 0
        cum_preds = []

        for i, x in enumerate(X_train):
            y = array(y_train)[i]
            with autograd.record():
                preds = model(X)[x]
                loss = cross_entropy(preds, y)
            loss.backward()
            trainer.step(1)

            cum_loss += loss.asscalar()
            cum_preds += [preds.asscalar()]

        feature_representations.append(features(X).asnumpy())
            
        if (e % (epochs//10)) == 0:
            print(f"Epoch {e}/{epochs} -- Loss: {cum_loss: .4f}")
            print(cum_preds)
    return feature_representations

def predict(model, X, nodes):
    preds = model(X)[nodes].asnumpy().flatten()
    return np.where(preds >= 0.5, 1, 0)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 20 µs


## Performance of Model 1

In [10]:
from sklearn.metrics import classification_report

feature_representations_1 = train(model_1, features_1, X_1, X_train, y_train, epochs=5000)
y_pred_1 = predict(model_1, X_1, X_test)
print(classification_report(y_test, y_pred_1))

Epoch 500/5000 -- Loss:  0.0001
[0.9999902, 4.623503e-05]
Epoch 1000/5000 -- Loss:  0.0000
[1.0, 8.968348e-10]
Epoch 1500/5000 -- Loss:  0.0000
[1.0, 1.7521857e-14]
Epoch 2000/5000 -- Loss:  0.0000
[1.0, 3.429584e-19]
Epoch 2500/5000 -- Loss:  0.0000
[1.0, 6.7126565e-24]
Epoch 3000/5000 -- Loss:  0.0000
[1.0, 1.313875e-28]
Epoch 3500/5000 -- Loss:  0.0000
[1.0, 2.5716606e-33]
Epoch 4000/5000 -- Loss:  0.0000
[1.0, 5.060105e-38]
Epoch 4500/5000 -- Loss:  0.0000
[1.0, 0.0]
Epoch 5000/5000 -- Loss:  0.0000
[1.0, 0.0]
              precision    recall  f1-score   support

       False       0.62      0.50      0.55        16
        True       0.58      0.69      0.63        16

   micro avg       0.59      0.59      0.59        32
   macro avg       0.60      0.59      0.59        32
weighted avg       0.60      0.59      0.59        32



## Performance of Model 2

In [11]:
feature_representations_2= train(model_2, features_2, X_2, X_train, y_train, epochs=250)
y_pred_2 = predict(model_2, X_2, X_test)
print(classification_report(y_test, y_pred_2))

Epoch 25/250 -- Loss:  1.4502
[0.46630818, 0.49708325]
Epoch 50/250 -- Loss:  1.4058
[0.48751295, 0.49711376]
Epoch 75/250 -- Loss:  1.3709
[0.5132726, 0.5053706]
Epoch 100/250 -- Loss:  1.3566
[0.5332241, 0.5170041]
Epoch 125/250 -- Loss:  1.3161
[0.5386774, 0.5021284]
Epoch 150/250 -- Loss:  1.1604
[0.5339666, 0.41315272]
Epoch 175/250 -- Loss:  0.8730
[0.5482581, 0.23811255]
Epoch 200/250 -- Loss:  0.5649
[0.625712, 0.09159255]
Epoch 225/250 -- Loss:  0.3007
[0.7656582, 0.033075843]
Epoch 250/250 -- Loss:  0.1270
[0.89338917, 0.014171573]
              precision    recall  f1-score   support

       False       0.83      0.94      0.88        16
        True       0.93      0.81      0.87        16

   micro avg       0.88      0.88      0.88        32
   macro avg       0.88      0.88      0.87        32
weighted avg       0.88      0.88      0.87        32

